In [1]:
import os
import sys
import time
import json
import shutil
import codecs
import requests
import psycopg2
import feedparser
import sqlalchemy
import urllib.parse

import numpy as np
import pandas as pd

from random import random
from datetime import datetime
from datetime import timezone
from selenium.webdriver import Chrome

sys.path.append("../")

from utils.sql_utils import *
from utils.scrape_utils import *
from utils.data_science_utils import *

In [ ]:
host=sql_utils.Host.LOCAL_JEAN
schema=models.schema
verbose=Verbose.INFO

In [ ]:
loadRoutine(host=host, schema=schema, verbose=verbose)

In [ ]:
pipelineNYTHeadlines(fetchSource=True, host=host, schema=schema, verbose=verbose)

In [ ]:
pipelineNYTNewsWire(fetchSource=True, verbose=Verbose.INFO) # startPage = 0

In [ ]:
pipelineNewsAPIHeadline(fetchSource=True)

driver.get("https://www.nytimes.com/")

Possible sources:

    "NYTimes": "https://www.nytimes.com/",
    "BBC": "https://www.bbc.co.uk/",
    "CNN": "https://edition.cnn.com/",
    "FoxNews": "https://www.foxnews.com/",
    "OAN": "https://www.oann.com/"

The Guardian, Yahoo news, Washington Post, Daily Mail, ...

https://www.4imn.com/top200/ for more

In [ ]:
all_sources = pd.read_sql("select * from newsdb.sources", getEngine())
all_sources.shape[0]

In [ ]:
all_articles = pd.read_sql_table("articles", getEngine())
all_articles.shape[0]

In [ ]:
all_article_contents = pd.read_sql_table("article_contents", getEngine())
all_article_contents.shape[0]

In [ ]:
sections = getNYTSections(source.api_key)

In [ ]:
createNlpModel(attributes=['title', 'description'])

In [ ]:
getSimilarArticlesFromText('VP pick', 'title', 10)

In [ ]:
getSimilarArticlesFromText('VP pick', 'description', 10)

In [ ]:
getSimilarArticlesFromText('Lebanon minister resigns in wake of deadly Beirut blast', 'title', 10)

In [2]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [143]:
url = "https://www.bbc.com/news/live/world-53775682"

In [147]:
url = "https://www.20min.ch/fr/story/avec-ses-propos-sur-son-remede-trump-cree-la-confusion-634629791483"

In [149]:
url = "https://nyti.ms/2GS6cZT"

In [150]:
title = scrapeArticleTitle(url)
title

'Trump Could Be Back in Public by Saturday. Experts Worry It’s Too Soon.'

In [99]:
q = getDBSession().query(
    models.Article.article_uuid, models.Article.title,
    models.Article.description, models.Article.article_url
).filter(
    models.Article.description.isnot(None)
).limit(10)

In [100]:
df = pd.read_sql(q.statement, q.session.bind)

In [151]:
df_titles = list(map(scrapeArticleTitle, list(df['article_url'])))
df_titles

['Travis Scott Announces ‘Tenet’ Single ‘The Plan’',
 'Portland police detain two suspects after reported shooting near federal courthouse protest site',
 'US consulate in Chengdu officially shuts in retaliation for Houston closure',
 'Hurricane Douglas menaces Hawaii coast, threatening heavy rain, strong winds',
 'Olivia de Havilland tributes: Jared Leto details unlikely friendship; Antonio Banderas, others also mourn',
 'US, China jockey for advantage in space race',
 "Mountain rescue crew carry Saint Bernard off England's highest peak",
 'Peter Green, 1960s guitar hero and co-founder of Fleetwood Mac, dies at 73',
 "Sean Shelby's Shoes: What's next for Darren Till after UFC on ESPN 14 loss?",
 "Biden campaign declines 'Fox News Sunday' interview, 1 week after Chris Wallace's Trump sitdown"]

In [ ]:
npr_feed_url = "https://feeds.npr.org/%d/rss.xml"

In [ ]:
for i in range(200, 50000):
    data = feedparser.parse(npr_feed_url % i)
    if 'feed' in data and 'title' in data['feed']:
        print("(\"" + (npr_feed_url % i) + "\", \"" + data['feed']['title'].replace(" : NPR", '') + "\"),")